# **Car Crashes Missing Data** 
<h5> In this notebook we will talk about "missing values problem". In fact dealing with missing values in a data set is an important step in data processing phase. We selected this data set coz it contains many missing values.</h5>

# <h3> Happy learning</h3>

![](https://img.freepik.com/free-vector/car-crash-concept-illustration_114360-7980.jpg)

In [ ]:
# Import libs

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Data Loading and Data Exploration

In [ ]:
data = pd.read_csv('/kaggle/input/car-crashes/Motor_Vehicle_Collisions_-_Crashes.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
# some statistical descriptive study
data.describe()

In [ ]:
data.dtypes.value_counts().plot.bar()

In [ ]:
# check the missing values 
plt.figure(dpi=120)
sns.heatmap(data.isna(),cbar=False)

<h5> Woooow missing data !!! How to fix this problem ??</h5>
<h5> From the above result we can see that we have many mssing values in our data set so we must deal with it.</h5>
<h5> There are several approaches to deal with missing data such as :</h5>
   <h6> * Remove rows with missing values.</h6>
   <h6> * Remove columns with a high percentage of missing values.</h6>
   <h6> * Imputation.</h6>
   


# 1) Remove rows with missing values
We will see if the number of missing values is small compared to the size of our data set so we can simply delete only rows with missing values. However, this approach can lead to loss of valuable data.

In [ ]:
 data_new = data.dropna()

In [ ]:
data_new

<h5> Oh!!! we have no more data we lost it because each row containt at least one missing value.</h5>
<h5>=> So this approach must be avoided in our case.</h5>

# 2) Remove columns with a high percentage of missing values
This approach aims to drop non critical columns with large proportion of missing values thus we will need to use the threshold concept. 
The thrshold is the number of missing values from which we can decide to remove a column or not.
We will use this formula to calculate the threshold: T = (M/100)*N 
   * M : The maximum percentage of missing values per comlumn we can tolorate. In this case we will choose 10%.
   * N : The total number of rows in our data set.
   * T : The threshold.

In [ ]:
# Threshold 
T = (10/100)*2018963
print("The threshold in our case is: ", T)

This means that any column with more than approximately 201,896 missing values are considered for removal.

In [ ]:
data.isnull().sum()

In [ ]:
# we will create a new dataset contains only columns with missing value < T
new_data = data[['CRASH DATE', 'CRASH TIME','NUMBER OF PERSONS INJURED','NUMBER OF PERSONS KILLED',
                 'NUMBER OF PEDESTRIANS INJURED','NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
                 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED','NUMBER OF MOTORIST KILLED', 
                 'CONTRIBUTING FACTOR VEHICLE 1','COLLISION_ID', 'VEHICLE TYPE CODE 1']]

In [ ]:
new_data.shape

In [ ]:
new_data.dtypes.value_counts()

In [ ]:
# we must change the dtype of CRASH DATE column
new_data['CRASH DATE'] = pd.to_datetime(new_data['CRASH DATE'])

In [ ]:
new_data.isnull().sum()

In [ ]:
new_data.isnull().sum().plot.bar()

In [ ]:
plt.figure(dpi=120)
sns.heatmap(new_data.isna(),cbar=False)

# 3) Imputation 
It involves filling in missing values with some estimated values. Common methods for imputation include:

* Mean/Median/Mode imputation: Replace missing values with the mean, median, or mode of the respective column.
* Interpolation: Estimate missing values based on the values of neighboring data points.
!!! BTW still there are many other imputation methods.
* Note: we will apply this method on the new data set 

# <h4>3.1) Mean/Median/Mode imputation</h4>

In [ ]:
# display the list of columns with empty values
null_list =[]
for col in new_data.columns:
    if (new_data[col].isnull().sum())>0:
        null_list.append(col)
        
null_list

# Categorical/ordinal columns
The mode imputation is the better choice for the categorical and ordinal variables.

In [ ]:
# we must put "include='all'" in the describe funtion in order to get the mode of the categorical columns.
# the top value is the mode.
new_data.describe(include='all')

In [ ]:
# also we can calculate the mode by this way 
cols = ['CONTRIBUTING FACTOR VEHICLE 1','VEHICLE TYPE CODE 1']
for col in cols:
    mode = new_data[col].mode()[0]
    print(mode)

In [ ]:
# also we can use this way to determin the mode, we take the value with the highest number of occurence 

new_data['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# in this columns we get "unspecified" value is the mode this is not logic to fill the empty cells with this values 
# thus we will use the value with second hight occurence 

In [ ]:
# As we can see in the from the previous table that the 'CONTRIBUTING FACTOR VEHICLE 1' contains NaN values 
# So we will fill it with first with the "Unspecified " value

new_data['CONTRIBUTING FACTOR VEHICLE 1'].fillna('Unspecified', inplace = True )

In [ ]:
# then we will replace the "Unspecified" value with "Driver Inattention/Distraction"
new_data['CONTRIBUTING FACTOR VEHICLE 1'] = new_data['CONTRIBUTING FACTOR VEHICLE 1'].replace('Unspecified','Driver Inattention/Distraction')

In [ ]:
new_data['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

In [ ]:
# still we have some strange values we have to fix them 
new_data['CONTRIBUTING FACTOR VEHICLE 1'] = new_data['CONTRIBUTING FACTOR VEHICLE 1'].replace('80','Driver Inattention/Distraction')
new_data['CONTRIBUTING FACTOR VEHICLE 1'] = new_data['CONTRIBUTING FACTOR VEHICLE 1'].replace('1','Driver Inattention/Distraction')

In [ ]:
new_data['VEHICLE TYPE CODE 1'].value_counts()

# for this column we will fill the empty values with "Sedan"

In [ ]:
# first we wil fill empty cells 
new_data['VEHICLE TYPE CODE 1'].fillna('Unspecified', inplace=True)

In [ ]:
# we  will replace the "Unspecified" values with "Sedan"
new_data['VEHICLE TYPE CODE 1'] = new_data['VEHICLE TYPE CODE 1'].replace('Unspecified', 'Sedan')

In [ ]:
# or we can simply do this 
new_data['VEHICLE TYPE CODE 1'].fillna(new_data['VEHICLE TYPE CODE 1'].mode(), inplace=True)

# Numerical columns

In [ ]:
list1 = ['NUMBER OF PERSONS INJURED','NUMBER OF PERSONS KILLED']
for l in list1:
    sns.distplot(new_data[l], hist = True )

* From this histogram we can see that the two columns are hight skewed to the right. 
* Also we can see from the statistical description table that the mean is greater than the median and the max value is greater than 0 witch representes the 75%. So we must use median imputation 
* The median is more suitable for ordinal or interval data with outliers or hight skewed distributions.
* It is more robust to outliers coz it less affected by extreme values compared to the mean 

In [ ]:
# check before th imputation 
new_data.isna().sum()

In [ ]:
new_data['NUMBER OF PERSONS INJURED'].fillna(new_data['NUMBER OF PERSONS INJURED'].median(), inplace = True )

In [ ]:
new_data['NUMBER OF PERSONS KILLED'].fillna(new_data['NUMBER OF PERSONS KILLED'].median(), inplace = True)

In [ ]:
new_data.dtypes.value_counts()

# <h4> 3.2) Interpolation imputation</h4>

In [ ]:
new_data2 = data[['CRASH DATE', 'CRASH TIME','NUMBER OF PERSONS INJURED','NUMBER OF PERSONS KILLED',
                 'NUMBER OF PEDESTRIANS INJURED','NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
                 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED','NUMBER OF MOTORIST KILLED', 
                 'CONTRIBUTING FACTOR VEHICLE 1','COLLISION_ID', 'VEHICLE TYPE CODE 1']]

In [ ]:
new_data2.head()

In [ ]:
new_data2.isnull().sum()

In [ ]:
new_data2['NUMBER OF PERSONS INJURED'] = new_data2['NUMBER OF PERSONS INJURED'].interpolate(method='linear', axis=0)


In [ ]:
new_data2['NUMBER OF PERSONS KILLED'] = new_data2['NUMBER OF PERSONS KILLED'].interpolate(method='nearest', axis=0)


* For nominal or categorical data the interpolation doesn't make sense coz there is no meaningful order or relationship between the categories. 
* Instead, we should use other imputation methods such as mode imputation to fill in the missing values as we have seen earlier or we can use binary fill as it is mentioned in the following cell.

In [ ]:
new_data2['VEHICLE TYPE CODE 1'].fillna(method='bfill', axis = 0, inplace= True)

In [ ]:
new_data2.isnull().sum()

# Hoppppe you find it helpfull !!!